In [1]:
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import combat_probability as cp
import tfb
from icepool import d

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [5]:
# Fig. 1: plots the damage distribution for an example combat encounter after two rounds of combat
combatants = [
    cp.Combatant(
        group='c1',
        name='c1',
        opponent='c2',
        level=5,
        hit_points = 30,
        damage = [2 @ cp.attack(5, 1 @ d(6) + 4, 2 @ d(6) + 4).damage(12)],
    ),
    cp.Combatant(
        group='c2',
        name='c2',
        opponent='c1',
        level=5,
        hit_points = 30,
        damage = [1 @ cp.attack(5, 1 @ d(8) + 5, 2 @ d(8) + 5).damage(12)],
    )
]
e = cp.Encounter(combatants, rounds=8)

# create figure
x_min = -1
#x_max = max([c.hit_points for c in combatants])
x_max = 61
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='damage',
            range=[x_min+0.5,x_max-0.5],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5.0),
        ),
        yaxis=dict(
            title_text='probability',
            #range=[0,0.1],
            tickformat='.0%'
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)
i = -1
for round in [1,2,3]:
    for group in ['c1']:
        i += 1
        damage = e.group_damage(group, e.get_turn(group, round), clip=False)
        x = np.linspace(-1, damage.max_outcome()+1, damage.max_outcome() + 3)
        y = np.array([damage.probability(d) for d in x])
        fig.add_trace(go.Scatter(
            x=x, 
            y=y,
            name=f'round {round:.0f}',
            line_shape='hvh',
            line_color=COLOR_LIST[i],
            mode='lines',
            fill='tozeroy',
            hoveron='points',
            hovertemplate=
                f'<b>round {round:.0f}</b><br>'+
                'damage %{x:.0f}<br>'+
                'probability %{y:.1%}<extra></extra>',
        ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-damage-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-damage-distribution-small')

In [6]:
# Fig. 2: plots the damage distribution for an example combat encounter after two rounds of combat
combatants = [
    cp.Combatant(
        group='combatant 1',
        name='combatant 1',
        opponent='combatant 2',
        level=5,
        hit_points = 30,
        damage = [2 @ cp.attack(5, 1 @ d(6) + 4, 2 @ d(6) + 4).damage(12)],
    ),
    cp.Combatant(
        group='combatant 2',
        name='combatant 2',
        opponent='combatant 1',
        level=5,
        hit_points = 30,
        damage = [1 @ cp.attack(5, 1 @ d(8) + 5, 2 @ d(8) + 5).damage(12)],
    )
]
e = cp.Encounter(combatants, rounds=8)

# create figure
x_min = -1
#x_max = max([c.hit_points for c in combatants])
x_max = 8
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='round',
            range=[x_min+0.5,x_max-0.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=5.0),
        ),
        yaxis=dict(
            title_text='probability',
            #range=[0,0.1],
            tickformat='.0%'
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)
colors = iter(COLOR_LIST)
for group in ['combatant 1']:
    color = next(colors)
    t = [0] + [t['turn'] for t in e.turns if t['group'] == group]
    x = [0] + [t['round'] for t in e.turns if t['group'] == group]
    y = e.group_damage_pdf(group)
    y = [0] + [y[i] for i in range(len(e.turns)) if e.turns[i]['group'] == group]
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=f'PDF',
        mode='lines',
        line_shape='hvh',
        line_color=color,
        fill='tozeroy',
        customdata=t,
        hoveron='points',
        hovertemplate=
            f'<b>PDF</b><br>'+
            'round %{x:.0f} (turn %{customdata:.0f})<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

    x = [0] + [t['round'] for t in e.turns if t['group'] == group]
    y = e.group_damage_cdf(group)
    y = [0] + [y[i] for i in range(len(e.turns)) if e.turns[i]['group'] == group]
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=f'CDF',
        line_color=color,
        customdata=t,
        hovertemplate=
            f'<b>CDF</b><br>'+
            'round %{x:.0f} (turn %{customdata:.0f})<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-dt-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-dt-distribution-small')

In [2]:
# Fig. 3: plots the damage distribution for an example combat encounter after two rounds of combat
combatants = [
    cp.Combatant(
        group='combatant 1',
        name='combatant 1',
        opponent='combatant 2',
        level=5,
        hit_points = 30,
        damage = [2 @ cp.attack(5, 1 @ d(6) + 4, 2 @ d(6) + 4).damage(12)],
    ),
    cp.Combatant(
        group='combatant 2',
        name='combatant 2',
        opponent='combatant 1',
        level=5,
        hit_points = 30,
        damage = [1 @ cp.attack(5, 1 @ d(8) + 5, 2 @ d(8) + 5).damage(12)],
    )
]
e = cp.Encounter(combatants, rounds=8)

print('mean', e.encounter_length(units='rounds'))
print('variance', e.encounter_length_variance(units='rounds'))
print('sigma', e.encounter_length_sigma(units='rounds'))

# create figure
x_min = -1
#x_max = max([c.hit_points for c in combatants])
x_max = 8
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='round',
            range=[x_min+0.5,x_max-0.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=5.0),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.02,1.02],
            tickformat='.0%'
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

colors = iter(COLOR_LIST)
for group in ['combatant 1','combatant 2']:
    color = next(colors)
    t = [0] + [t['turn'] for t in e.turns if t['group'] == group]
    x = [0] + [t['round'] for t in e.turns if t['group'] == group]
    y = e.group_win_pdf(group)
    y = [0] + [y[i] for i in range(len(e.turns)) if e.turns[i]['group'] == group]
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        legendgroup=group,
        legendgrouptitle_text=group,
        name=f'PDF',
        mode='lines',
        line_shape='hvh',
        line_color=color,
        fill='tozeroy',
        customdata=t,
        hoveron='points',
        hovertemplate=
            f'<b>{group} - PDF</b><br>'+
            'round %{x:.0f} (turn %{customdata:.0f})<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

    x = [0] + [t['round'] for t in e.turns if t['group'] == group]
    y = e.group_win_cdf(group)
    y = [0] + [y[i] for i in range(len(e.turns)) if e.turns[i]['group'] == group]
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=f'CDF',
        legendgroup=group,
        legendgrouptitle_text=group,
        line_color=color,
        customdata=t,
        hovertemplate=
            f'<b>{group} - CDF</b><br>'+
            'round %{x:.0f} (turn %{customdata:.0f})<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

    print(f'{group} - {e.group_damage_mean(group, clip=True):.1f}')

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-win-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-win-distribution-small')

mean 2.754796213908374
variance 0.7115691975433006
sigma 0.8435456108256983
combatant 1 - 29.3
combatant 2 - 15.5


In [8]:
# Fig. 4: plots the final damage distribution for an example combat encounter
combatants = [
    cp.Combatant(
        group='combatant 1',
        name='combatant 1',
        opponent='combatant 2',
        level=5,
        hit_points = 30,
        damage = [2 @ cp.attack(5, 1 @ d(6) + 4, 2 @ d(6) + 4).damage(12)],
    ),
    cp.Combatant(
        group='combatant 2',
        name='combatant 2',
        opponent='combatant 1',
        level=5,
        hit_points = 30,
        damage = [1 @ cp.attack(5, 1 @ d(8) + 5, 2 @ d(8) + 5).damage(12)],
    )
]
e = cp.Encounter(combatants, initiative=['combatant 1','combatant 2'], rounds=8)

# create figure
x_min = -1
x_max = max([c.hit_points for c in combatants]) + 2
#x_max = 100
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='final damage',
            #range=[x_min+0.5,x_max-0.5],
            range=[-0.5,31.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1.0),
        ),
        yaxis=dict(
            title_text='probability',
            range=[0,0.11],
            tickformat='.0%'
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)
colors = iter(COLOR_LIST)
color = next(colors)
for group in ['combatant 2']:
    color = next(colors)
    #damage = e.group_damage(group, 6)
    damage = e.group_damage_distribution(group, clip=True)
    print(group, damage.mean(), damage.sd())
    d_mean = e.group_damage_mean(group, method='approx', clip=True)
    d_sigma = e.group_damage_sigma(group, method='approx', clip=True)
    print(group, d_mean, d_sigma)
    x = np.linspace(-1, damage.max_outcome()+1, damage.max_outcome() + 3)
    y = np.array([damage.probability(d) for d in x])
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=f'{group}',
        line_shape='hvh',
        line_color=color,
        fill='tozeroy',
        hoveron='points',
        hovertemplate=
            f'<b>{group}</b><br>'+
            'damage %{x:.0f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-final-damage-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-final-damage-distribution-small')

combatant 2 15.496107504175148 8.832251054933918
combatant 2 15.540309054475653 8.726940408256919


In [10]:
# Fig. 5: plots the damage distribution for an example combat encounter after two rounds of combat
combatants = [
    cp.Combatant(
        group='c1',
        name='c1',
        opponent='c2',
        level=5,
        hit_points = 30,
        damage = [2 @ cp.attack(5, 1 @ d(6) + 4, 2 @ d(6) + 4).damage(12)],
    ),
    cp.Combatant(
        group='c2',
        name='c2',
        opponent='c1',
        level=5,
        hit_points = 30,
        damage = [1 @ cp.attack(5, 1 @ d(8) + 5, 2 @ d(8) + 5).damage(12)],
    )
]
e = cp.Encounter(combatants, rounds=8)

# create figure
x_min = -1
#x_max = max([c.hit_points for c in combatants])
x_max = 61
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='damage',
            range=[x_min+0.5,x_max-0.5],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5.0),
        ),
        yaxis=dict(
            title_text='probability',
            #range=[0,0.1],
            tickformat='.0%',
            tick0=0, dtick=0.02,
            minor=dict(tick0=0, dtick=0.01),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)
i = -1
for round in [1,2,3]:
    for group in ['c1']:
        i += 1
        damage = e.group_damage(group, e.get_turn(group, round), clip=False)
        x = np.linspace(-1, damage.max_outcome()+1, damage.max_outcome() + 3)
        y = np.array([damage.probability(d) for d in x])
        fig.add_trace(go.Scatter(
            x=x, 
            y=y,
            #name=f'damage distribution',
            name=f'round {round:.0f}',
            legendgroup=f'round {round:.0f}',
            legendgrouptitle_text=None,
            line_shape='hvh',
            line_color=COLOR_LIST[i],
            mode='lines',
            fill='tozeroy',
            hoveron='points',
            hovertemplate=
                f'<b>round {round:.0f}</b><br>'+
                'damage %{x:.0f}<br>'+
                'probability %{y:.1%}<extra></extra>',
        ))
        #_, r2 = gaussian_fit(damage)
        #print(f'{round:.0f} - R squared = {r2:.4f}')
        
        damage = cp.gaussian_die_approximation(damage)
        x = np.linspace(-1, damage.max_outcome()+1, damage.max_outcome() + 3)
        y = np.array([damage.probability(d) for d in x])
        fig.add_trace(go.Scatter(
            x=x, 
            y=y,
            #name=f'Gaussian fit',
            legendgroup=f'round {round:.0f}',
            #legendgrouptitle_text=None,
            showlegend=False,
            line_color=COLOR_LIST[i],
            mode='lines',
            hovertemplate=
                f'<b>round {round:.0f}</b><br>'+
                'damage %{x:.0f}<br>'+
                'probability %{y:.1%}<extra></extra>',
        ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-gaussian-fit-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-gaussian-fit-small')

In [13]:
# plots the number of turns it takes for a save to be sufficiently Gaussian
from icepool import d
import numpy as np

"""
0.99    55.99 + -68.24*phit
0.95    16.79 + -18.44*phit
"""

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='chance to hit',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.05),
        ),
        yaxis=dict(
            title_text='saves',
            #type='log',
            range=[0,20],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
            #tickformat='.0%'
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


for roll in [4, 6, 8, 10, 12]:
    pc = cp.baseline_pc(16)
    x = []
    y = []
    for ab in list(range(0, 16)):
        dc = ab + 8 + 2
        dpr = cp.save(dc, 1 @ d(roll), 0.5)
        dmg = dpr.damage(pc['save bonus'], 1)
        rnd = cp.rounds_to_gaussian(dmg, rsquared=0.99, method='top-half')

        #p_miss = dmg.probability(0)
        #p_hit = 1 - p_miss
        p_hit = 0.05*(dc - pc['save bonus'] - 1)
        x.append(p_hit)
        y.append(rnd)
        #print(f'{p_hit:.2f}, {rnd:.0f}')

    coefs = np.polyfit(x, np.log10(y), 1)
    poly = np.poly1d(coefs)
    print(f'{coefs[1]:.2f} + {coefs[0]:.2f}*phit')

    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=f'd{roll:.0f}',
        hovertemplate=
            f'<b>d{roll:.0f}</b><br>'+
            'hit %{x:.0%}<br>'+
            'attacks %{y:.0f}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, sformat='large', name=f'./fig-simple-hits-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-simple-hits-small')

0.84 + -0.47*phit
1.08 + -0.74*phit
1.16 + -0.83*phit
1.24 + -0.99*phit
1.23 + -0.97*phit


In [16]:
# Fig. 6: plots the number of turns it takes for an attack to be sufficiently Gaussian

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='chance to hit',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.05),
        ),
        yaxis=dict(
            title_text='attacks',
            #type='log',
            range=[0,20],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
            #tickformat='.0f'
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

r2 = 0.95
m = 1
#roll = 12
#for method in ['save','attack']:
method = 'attack'
die_sizes = [4, 6, 8, 10, 12]
for roll in die_sizes:
    pc = cp.baseline_pc(16)
    x = []
    y = []
    for ab in list(range(0, 16)):
        if method == 'attack':
            dpr = cp.attack(ab, m @ d(roll), (2*m) @ d(roll))
            dmg = dpr.damage(pc['armor class'])
            rnd = cp.rounds_to_gaussian(dmg, rsquared=r2, method='range')
            p_miss = dmg.probability(0)
            p_hit = 1 - p_miss
        else:
            dc = ab + 8 + 2
            dpr = cp.save(dc, m @ d(roll), 0.5)
            dmg = dpr.damage(pc['save bonus'], 1)
            rnd = cp.rounds_to_gaussian(dmg, rsquared=r2, method='range')
            p_hit = 0.05*(dc - pc['save bonus'] - 1)

        x.append(p_hit)
        y.append(rnd)
        #print(f'{p_hit:.2f}, {rnd:.0f}')

    coefs = np.polyfit(x, np.log10(y), 1)
    poly = np.poly1d(coefs)
    #print(f'{coefs[1]:.2f} + {coefs[0]:.2f}*phit')

    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=f'd{roll:.0f}',
        hovertemplate=
            f'<b>d{roll:.0f}</b><br>'+
            'hit %{x:.0%}<br>'+
            'attacks %{y:.0f}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-attacks-for-gaussian-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-attacks-for-gaussian-small')

In [17]:
# Fig. 7: plots the damage distribution for an example combat encounter after two rounds of combat

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='rounds',
            range=[-0.5, 7.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=5.0),
        ),
        yaxis=dict(
            title_text='probability',
            #range=[0,0.1],
            tickformat='.0%'
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

dpr1 = 1 @ cp.attack(5, 1 @ d(6) + 4, 2 @ d(6) + 4).damage(12)
hp1 = 30

dpr2 = 1 @ cp.attack(5, 1 @ d(8) + 5, 2 @ d(8) + 5).damage(12)
hp2 = 15

# original
combatants = [
    cp.Combatant(
        group='combatant 1',
        name='combatant 1',
        opponent='combatant 2',
        level=5,
        hit_points = hp1,
        damage = [dpr1],
    ),
    cp.Combatant(
        group='combatant 2',
        name='combatant 2',
        opponent='combatant 1',
        level=5,
        hit_points = hp2,
        damage = [dpr2],
    )
]
e = cp.Encounter(combatants, rounds=8)

colors = iter(COLOR_LIST)
for method, lgroup in [('exact','exact'),('approx','Gaussian')]:
    for group in ['combatant 1']:
        color = next(colors)
        
        t = [0] + [t['turn'] for t in e.turns if t['group'] == group]
        x = [0] + [t['round'] for t in e.turns if t['group'] == group]
        y = e.group_damage_pdf(group, method=method)
        y = [0] + [y[i] for i in range(len(e.turns)) if e.turns[i]['group'] == group]
        fig.add_trace(go.Scatter(
            x=x, 
            y=y,
            name=f'PDF',
            legendgroup=lgroup,
            legendgrouptitle_text=lgroup,
            mode='lines',
            line_shape='hvh',
            line_color=color,
            fill='tozeroy',
            customdata=t,
            hoveron='points',
            hovertemplate=
                f'<b>{lgroup} - PDF</b><br>'+
                'round %{x:.0f} (turn %{customdata:.0f})<br>'+
                'probability %{y:.1%}<extra></extra>',
        ))
        print(method, sum(np.array(x)*np.array(y)))

        t = [0] + [t['turn'] for t in e.turns if t['group'] == group]
        x = [0] + [t['round'] for t in e.turns if t['group'] == group]
        y = e.group_damage_cdf(group, method=method)
        y = [0] + [y[i] for i in range(len(e.turns)) if e.turns[i]['group'] == group]
        fig.add_trace(go.Scatter(
            x=x, 
            y=y,
            name=f'CDF',
            legendgroup=lgroup,
            legendgrouptitle_text=lgroup,
            line_color=color,
            customdata=t,
            hovertemplate=
                f'<b>{lgroup} - CDF</b><br>'+
                'round %{x:.0f} (turn %{customdata:.0f})<br>'+
                'probability %{y:.1%}<extra></extra>',
        ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-gaussian-dt-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-gaussian-dt-distribution-small')

exact 3.331591977974537
approx 3.3908433397702415
